In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [70]:
# 패키지 import
import tarfile
import os
import pandas as pd
import numpy as np
import random
import math

#!pip install selenium

#!apt-get update

#!apt install chromium-chromedriver

!pip install catboost

from selenium import webdriver
from bs4 import BeautifulSoup
import time
from urllib.request import urlopen
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import sys

import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
#from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 78 kB/s 


In [85]:
data = pd.read_csv("/content/drive/MyDrive/암마커추출/data.csv", sep=',',low_memory=False)
data

,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,?|280660,?|317712,...,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009,psiTPTE22|387590,tAKR|389932,Cancer_info
0,0.0,10.0113,11.2820,49.5994,848.9397,0.0,345.2308,1.0472,0.0000,0.0,...,292.5212,959.2460,0.6981,1088.0531,2837.9440,871.2802,575.2683,6.6323,0.0000,0
1,0.0,7.1957,12.4436,90.5117,924.0158,0.0,145.2025,1.6098,0.0000,0.0,...,493.5597,1179.3275,20.2833,787.5061,2351.2500,1138.1170,690.2752,179.9738,0.0000,0
2,0.0,7.2453,6.0184,49.5366,1140.6781,0.0,51.7284,0.0000,0.0000,0.0,...,365.4149,843.9028,26.5274,475.1720,5437.4534,1170.5214,532.8691,6.3003,0.0000,0
3,0.0,11.3311,7.5740,82.8303,807.1729,0.0,240.0221,0.4786,0.2393,0.0,...,346.7517,946.6872,64.8514,908.1593,6770.1537,1169.2401,663.8297,35.1777,0.0000,0
4,0.0,3.2254,3.4942,72.5351,562.0037,0.0,274.2822,0.6109,0.0000,0.0,...,563.2254,1320.7086,7.9414,778.8638,3341.4783,1737.3244,723.2743,378.1307,0.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005,0.0,19.9503,47.1026,176.7177,1188.3278,0.0,226.8212,1.6556,0.0000,0.0,...,507.4503,1392.7980,126.2417,580.2980,2681.7053,1069.5364,783.1126,5.3808,1.2417,1
1006,0.0,30.0872,15.2957,188.7215,1248.0303,0.0,147.4945,0.4202,0.0000,0.0,...,366.8453,1326.6099,6.3032,845.8872,6094.3376,663.9353,778.6532,2.1011,0.0000,1
1007,0.0,53.6593,33.3907,260.3332,789.3606,0.0,854.3794,0.0000,0.0000,0.0,...,336.9156,2002.6867,101.5583,613.6486,2560.9887,730.2526,761.9559,12.3589,0.0000,1
1008,0.0,72.5666,42.1832,160.1624,460.8626,0.0,569.2226,1.3312,0.0000,0.0,...,584.1321,2074.5474,150.4260,1155.2183,1185.0373,833.3333,1383.1203,1.0650,0.0000,1


In [78]:
def modeling_run(data, method):
  model_name =  {"RF" : RandomForestClassifier(), "LGBM" : LGBMClassifier(), "XGB" : XGBClassifier(), "CAT" : CatBoostClassifier(), "Ada" : AdaBoostClassifier()}
  model_final = [keys for keys in method]

  X_features = data.iloc[:, :-1]
  y_target = data.iloc[:, -1]

  X_train, X_test, Y_train, Y_test = train_test_split(X_features, y_target, 
                                                      test_size=0.2, random_state=97, 
                                                      stratify=y_target)
  
  modeling_df = pd.DataFrame()
  for model in model_final:
    model_run = model_name[model]
    model_run.fit(X_train, Y_train)
    importances = model_run.feature_importances_
    #sorted = importances.argsort()[::-1]

    top_biomarker = pd.DataFrame(importances, index=X_train.columns, columns=["{}".format(model)])
    modeling_df = pd.concat([modeling_df, top_biomarker], axis=1)

  return modeling_df

topBio = modeling_run(data, ["RF", "XGB"])
topBio

,RF,XGB
Hybridization REF,,
?|100130426,0.000000,0.000000
?|100133144,0.000000,0.000000
?|100134869,0.000049,0.000000
?|10357,0.000000,0.000000
?|10431,0.000179,0.000000
...,...,...
ZYX|7791,0.000000,0.003154
ZZEF1|23140,0.000000,0.000000
ZZZ3|26009,0.000000,0.000000
